In [1]:
import torch
import torchvision

print(torch.__version__)

print(torchvision.__version__)


2.3.1+cu121
0.18.1+cu121


In [2]:
from torchvision.datasets import CIFAR10

In [3]:
targets = [1, 2, 8, 9]

In [4]:
from torchvision import transforms

train_transforms = transforms.Compose([
  transforms.Resize((224, 224)),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize(
      (0.4914, 0.4822, 0.4465),
      (0.2023, 0.1994, 0.2010))])

train_data = CIFAR10(root="./train/",
                    train=True,
                    download=True,
                    transform=train_transforms)

Files already downloaded and verified


In [5]:
indices = [i for i, label in enumerate(train_data.targets) if label in targets]

In [6]:
from torch.utils.data.dataset import Subset
train_subset = Subset(train_data, indices)

In [7]:
trainloader = torch.utils.data.DataLoader(
                    train_subset,
                    batch_size=16,
                    shuffle=True)

In [8]:
test_transforms = transforms.Compose([
  transforms.Resize((224, 224)),
  transforms.ToTensor(),
  transforms.Normalize(
      (0.4914, 0.4822, 0.4465),
      (0.2023, 0.1994, 0.2010))])

test_data = torchvision.datasets.CIFAR10(
      root="./test/",
      train=False,
      download=True,
      transform=test_transforms)


Files already downloaded and verified


In [9]:
indices = [i for i, label in enumerate(test_data.targets) if label in targets]

In [10]:
test_subset = Subset(test_data, indices)

In [11]:
testloader = torch.utils.data.DataLoader(
                    test_subset,
                    batch_size=16,
                    shuffle=False)

In [12]:
data_batch, labels_batch = next(iter(trainloader))
print(data_batch.size())

print(labels_batch.size())

torch.Size([16, 3, 224, 224])
torch.Size([16])


In [14]:
!pip install torchinfo

In [15]:
from torchvision.models.vgg import vgg16
import torchvision.models as models
from torchinfo import summary


vgg16 = vgg16(weights= models.VGG16_Weights.DEFAULT)

In [16]:
len(list(vgg16.children()))

3

In [17]:
print(list(vgg16.children()))

[Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_s

In [18]:
print(vgg16.features)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [19]:
print(vgg16.avgpool)

AdaptiveAvgPool2d(output_size=(7, 7))


In [20]:
print(vgg16.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)


In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = vgg16.to(device)

In [22]:
summary(model, input_size=(16, 3, 224, 224), row_settings=("depth", "ascii_only"))

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [16, 1000]                --
+ Sequential: 1-1                        [16, 512, 7, 7]           --
|    + Conv2d: 2-1                       [16, 64, 224, 224]        1,792
|    + ReLU: 2-2                         [16, 64, 224, 224]        --
|    + Conv2d: 2-3                       [16, 64, 224, 224]        36,928
|    + ReLU: 2-4                         [16, 64, 224, 224]        --
|    + MaxPool2d: 2-5                    [16, 64, 112, 112]        --
|    + Conv2d: 2-6                       [16, 128, 112, 112]       73,856
|    + ReLU: 2-7                         [16, 128, 112, 112]       --
|    + Conv2d: 2-8                       [16, 128, 112, 112]       147,584
|    + ReLU: 2-9                         [16, 128, 112, 112]       --
|    + MaxPool2d: 2-10                   [16, 128, 56, 56]         --
|    + Conv2d: 2-11                      [16, 256, 56, 56]         29

We donn't see flatten layer in torch info as flaten layer has not trainable parametrs and isn't trained

In [23]:
summary(model, input_size=(16,3,32,32), row_settings=("depth", "ascii_only"))

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [16, 1000]                --
+ Sequential: 1-1                        [16, 512, 1, 1]           --
|    + Conv2d: 2-1                       [16, 64, 32, 32]          1,792
|    + ReLU: 2-2                         [16, 64, 32, 32]          --
|    + Conv2d: 2-3                       [16, 64, 32, 32]          36,928
|    + ReLU: 2-4                         [16, 64, 32, 32]          --
|    + MaxPool2d: 2-5                    [16, 64, 16, 16]          --
|    + Conv2d: 2-6                       [16, 128, 16, 16]         73,856
|    + ReLU: 2-7                         [16, 128, 16, 16]         --
|    + Conv2d: 2-8                       [16, 128, 16, 16]         147,584
|    + ReLU: 2-9                         [16, 128, 16, 16]         --
|    + MaxPool2d: 2-10                   [16, 128, 8, 8]           --
|    + Conv2d: 2-11                      [16, 256, 8, 8]           29

No the output dimensions of all layers are identical in both summaries 

After the Adaptive average pooling layer the output dimensions are same

The reason is because of adaptive average pool layer as it produces a fixed out no matter what the input dimensions are by design
The first layer is adaptive average pooling layer

In [24]:
for submodel in vgg16.children():
    print(submodel.children)

<bound method Module.children of Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

Yes the submodel type is sequential 

In [25]:
def modify_vgg16_classifier(num_classes=4):
    model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)

    for param in model.features.parameters():
        param.requires_grad = False


    model.classifier = torch.nn.Sequential(
        torch.nn.Linear(512*7*7, 4096),
        torch.nn.ReLU(True),
        torch.nn.Dropout(),
        torch.nn.Linear(4096, 4096),
        torch.nn.ReLU(True),
        torch.nn.Dropout(),
        torch.nn.Linear(4096, num_classes)
    )

    return model

modified_vgg16 = modify_vgg16_classifier(num_classes=4)


In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"
modified_model_1 = modified_vgg16.to(device)

In [27]:
summary(modified_model_1, input_size=(16, 3, 224, 224), row_settings=("depth", "ascii_only"))

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [16, 4]                   --
+ Sequential: 1-1                        [16, 512, 7, 7]           --
|    + Conv2d: 2-1                       [16, 64, 224, 224]        (1,792)
|    + ReLU: 2-2                         [16, 64, 224, 224]        --
|    + Conv2d: 2-3                       [16, 64, 224, 224]        (36,928)
|    + ReLU: 2-4                         [16, 64, 224, 224]        --
|    + MaxPool2d: 2-5                    [16, 64, 112, 112]        --
|    + Conv2d: 2-6                       [16, 128, 112, 112]       (73,856)
|    + ReLU: 2-7                         [16, 128, 112, 112]       --
|    + Conv2d: 2-8                       [16, 128, 112, 112]       (147,584)
|    + ReLU: 2-9                         [16, 128, 112, 112]       --
|    + MaxPool2d: 2-10                   [16, 128, 56, 56]         --
|    + Conv2d: 2-11                      [16, 256, 56, 56]   

In [28]:
import time
from tqdm import tqdm

def train_model(model, train_loader, loss_fn, optimizer, epochs=1, device='cuda',targets=[1, 2, 8, 9]):
    labmap = {x: i for i, x in enumerate(targets)}
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 10)
        model.train()
        # Wrap the dataloader with tqdm
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}', leave=False):
            inputs = inputs.to(device)
            labels = labels.to(device)
            new_labels = torch.tensor([labmap[label.item()] for label in labels])
            new_labels = new_labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = loss_fn(outputs, new_labels)
                loss.backward()
                optimizer.step()
    return model


In [29]:
import torch.nn as nn
import torch.optim as optim
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [30]:
modified_model_1 = train_model(modified_model_1,trainloader,loss_fn, optimizer, epochs=10, device=device)

Epoch 1/10
----------


Epoch 2/10
----------


Epoch 3/10
----------


Epoch 4/10
----------


Epoch 5/10
----------


Epoch 6/10
----------


Epoch 7/10
----------


Epoch 8/10
----------


Epoch 9/10
----------


Epoch 10/10
----------


In [31]:
num_correct = 0
for x_test_batch, y_test_batch in testloader:
  modified_model_1.eval()
  y_test_batch = y_test_batch.to(device)
  x_test_batch = x_test_batch.to(device)
  y_pred_batch = modified_model_1(x_test_batch)
  _, predicted = torch.max(y_pred_batch, 1)
  num_correct += (predicted == y_test_batch).float().sum()
accuracy = num_correct/(len(testloader)*testloader.batch_size)
print(len(testloader), testloader.batch_size)
print("Test Accuracy Modified Vgg 1 : {}".format(accuracy))

250 16
Test Accuracy Modified Vgg 1 : 0.08575000613927841


In [32]:
vgg16 = models.vgg16()

In [33]:
num_correct = 0
for x_test_batch, y_test_batch in testloader:
  vgg16.eval()
  y_test_batch = y_test_batch.to(device)
  x_test_batch = x_test_batch.to(device)
  y_pred_batch = model(x_test_batch)
  _, predicted = torch.max(y_pred_batch, 1)
  num_correct += (predicted == y_test_batch).float().sum()
  accuracy = num_correct/(len(testloader)*testloader.batch_size)
print(len(testloader), testloader.batch_size)
print("Test AccuracyVgg : {}".format(accuracy))

250 16
Test AccuracyVgg : 0.0


In [34]:
def modify_vgg16_classifier1(num_classes=4):
    model = models.vgg16(weights=models.efficientnet_b0)

    for param in model.features.parameters():
        param.requires_grad = False


    model.classifier = torch.nn.Sequential(
        torch.nn.Linear(512*7*7, 4096),
        torch.nn.ReLU(True),
        torch.nn.Dropout(),
        torch.nn.Linear(4096, 4096),
        torch.nn.ReLU(True),
        torch.nn.Dropout(),
        torch.nn.Linear(4096, num_classes)
    )

    return model

In [38]:
modified_model_2 = modify_vgg16_classifier1(4)
modified_model_2 = modified_model_2.to(device)

In [39]:
modified_model_2 = train_model(modified_model_2,trainloader,loss_fn, optimizer, epochs=10, device=device)

Epoch 1/10
----------


Epoch 2/10
----------


Epoch 3/10
----------


Epoch 4/10
----------


Epoch 5/10
----------


Epoch 6/10
----------


Epoch 7/10
----------


Epoch 8/10
----------


Epoch 9/10
----------


Epoch 10/10
----------


In [40]:
num_correct = 0
for x_test_batch, y_test_batch in testloader:
  modified_model_2.eval()
  y_test_batch = y_test_batch.to(device)
  x_test_batch = x_test_batch.to(device)
  y_pred_batch = modified_model_2(x_test_batch)
  _, predicted = torch.max(y_pred_batch, 1)
  num_correct += (predicted == y_test_batch).float().sum()
accuracy = num_correct/(len(testloader)*testloader.batch_size)
print(len(testloader), testloader.batch_size)
print("Test Accuracy Modified Vgg 2 : {}".format(accuracy))

250 16
Test Accuracy Modified Vgg 2 : 0.01850000023841858
